# Vektorimuotoinen paikkatieto

Vektoriaineistot ovat paikkatietoa joko pisteinä viivointa tai monikulmioina. Tässä osiossa tutustutaan vektoridatan käsittelyn perusteisiin Pythonissa.

# Shapely ja geometriat

**Shapely** on vektoriaineistojen kannalta olennainen kirjasto, sillä se toteuttaa tuen geometrioiden luontiin, esittämiseen ja käsittelyyn. GeoPandasin geometriat perustuvat täysin Shapelyyn, joten, vaikka shapelyä ei välttämättä käytettäisi suoraan, on hyvä ymmärtää kirjaston perusteet.

Aloitetaan ottamalla käyttöön Shapely-kirjastosta pisteet, viivat ja monikulmiot.

In [ ]:
from shapely import Point, LineString, Polygon

Voimme nyt luoda pisteen, eli **Point**-olion. Huomaa myös iso P olitota luodessa - Pythonissa *konstruktorit*, eli olioita luovat metodit, kirjoitetaan usein käyttämällä CamelCase-notaatiota.

In [ ]:
point = Point(1, 2)
point

In [ ]:
print(point)

Voimme ottaa mukaan myös kolmannen ulottuvuuden:

In [ ]:
point_3d = Point(2, 4, 1)
point_3d

Tehdään seuraavaksi monta pistettä, ja muodostetaan niistä viiva (**LineString**-olio):

In [ ]:
point_1 = Point(0, 0)
point_2 = Point(-1, 1)
point_3 = Point(0.5, 1.5)
point_4 = Point(1, 0.5)

list_of_points = [point_1, point_2, point_3, point_4]

line = LineString(list_of_points)
line

Shapely-geometrioilla on monia hyödyllisiä attribuutteja. Tässä esimerkiksi viivan pituus ja keskipiste.

In [ ]:
print(line.length)
print(line.centroid)

Monikulmion luonti onnistuu samaan tapaan:

In [ ]:
polygon = Polygon(list_of_points)
polygon

Lisää esimerkkejä metodeista. Lasketaan keskipiste ja alue, ja luodaan monikulmiolle bufferi:

In [ ]:
print(polygon.centroid)
print(polygon.area)
polygon.buffer(1)

Shapely tukee myös multi -versioita kaikista geometriatyypeistä:

In [ ]:
from shapely import MultiPoint, MultiLineString, MultiPolygon

multipoint = MultiPoint(
    [Point(1, 2), Point(3, 1)]
)
multipoint

In [ ]:
multiline = MultiLineString(
    [LineString([(1, 7), (6, 4)]), LineString([(0, -1), (-4, 5)])]
)
multiline

In [ ]:
multipolygon = MultiPolygon(
    [Polygon([(0, 0), (-10, 0), (2, 20)]), Polygon([(3, 6), (10, 6), (11, 20)])]
)
multipolygon

## Harjoitus - ympyrä

Tee Shapely-geometrioita ja metodeja hyödyntäen "ympyrä".
1. Vinkki: et tarvitse yhtäkään uutta `import`-komentoa.
2. Vinkki: tämä on hieman kompakysymys. Helpompaa voi olla ajatella: mitä geometriatyyppiä ja metodia käyttämällä teen mahdollisimman pyöreän monikulmion?

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
my_buffer = Point(1,1).buffer(10)

print(type(my_buffer))
my_buffer

# GeoPandas

Seuraavaksi siirrytään vektoriaineistojen käsittelyssä erittäin monikäyttöiseen **GeoPandas**-kirjastoon. GeoPandas on aktiivisesti ylläpidetty ja kehitetty, laajalti käytetty kirjasto, joka on vakiinnuttanut asemansa paikkatietoalalla. Se tukee lähes mitä vain vektorimuotoisia paikkatietoformaatteja, sekä toimii yhteen esim. PostGISin kanssa.

# GeoDataFrame ja GeoSeries

GeoPandasissa keskeisin tietorakenne on **GeoDataFrame**. Se on muuten aivan kuin Pandasin DataFrame, mutta tukee geometrioita. GeoDataFramessa on ainakin yksi sarake, joka pitää sisällään jokaisen rivin geometriatiedon, eli vaikkapa pisteen, viivan tai monikulmion. Tämä sarake on tyypiltään **GeoSeries**. GeoDataframe ja GeoSeries toteuttavat monenlaisia paikkatietoanalyysejä ja datan käsittelyä mahdollistavia metodeja. Tässä materiaalissa ehditään tekemään vain pintaraapaisu - laajemman katasauksen löydät vaikkapa [GeoPandasin omasta dokumentaatiosta](https://geopandas.org/en/stable/docs/reference.html).

Luodaan seuraavaksi GeoDataFrame tilastokeskuksen kunta-aineistosta (`./data/kunnat.gpkg`):

In [ ]:
from pathlib import Path
import geopandas as gpd

In [ ]:
file_path = Path("./data/kunnat.gpkg")

municipalities = gpd.read_file(file_path)

In [ ]:
municipalities.head()

In [ ]:
type(municipalities)

Geometriatiedot ovat oletuksena sarakkeessa nimeltä "geometry". GeoDataFramen geometrioihin päästään käsiksi `geometry`-attribuutilla:

In [ ]:
municipalities.geometry

Jos tiedämme geometriasarakkeen nimen, voimme käyttää myös sitä:

In [ ]:
municipalities["geometry"]

Huomaa, että geometriasarakkeen tyyppi on **GeoSeries**. GeoDataFramen muut sarakkeet ovat pandas Series.

In [ ]:
type(municipalities["geometry"])

In [ ]:
type(municipalities["nimi"])

Tarkastellaan vielä yksittäistä GeoSeriesin alkiota (tässä tapauksessa ensimmäisen rivin geometriaa). Pohjimmiltaan geometriat ovat shapely-olioita:

In [ ]:
my_geom = municipalities["geometry"].iloc[0]

my_geom

In [ ]:
type(my_geom)

Myös muut pandasista tutut valinnat toimivat myös GeoPandasissa. Valitaan vaikkapa rivejä "nimi"-sarakkeen perusteella:

In [ ]:
parainen = municipalities.loc[municipalities["nimi"] == "Parainen"]
parainen

# GeoDataFrame kartaksi

Samaan tapaan kuin DataFramen, voimme helposti visualisoida myös GeoDataFramen sen `plot`-metodilla. Myös geopandas totetuttaa tämän toiminnallisuuden Matplotlib-kirjaston avulla.

In [ ]:
municipalities.plot()

## Harjoitus - kunnan kartta

Visualisoi yksittäinen kunta `plot`-metodilla.

1. Valitse kunta esimerkiksi nimen perustella
2. Käytä valinnasta saadun GeoDataFramen plot-metodia

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
my_municipality = municipalities.loc[municipalities["nimi"] == "Parainen"]
my_municipality.plot()

## Harjoitus - interaktiivinen kartta

Staattisten Matplotlib-pohjaisten karttojen lisäksi voit tutkia GeoDataFramea interaktiivisesti `explore`-metodilla. Tämä toiminnallisuus käyttää taustalla **Folium**-kirjastoa.

1. Visualisoi edellisessä harjoituksessa valitsemasi kunta `explore`-metodilla.

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
my_municipality.explore()

# Koordinaattijärjestelmät

Paikkatietoanalyyseissä ensiarvoisen tärkeää ovat **koordinaattijärjestelmät**. GeoDataFramen koordinaattijärjestelmään pääsee käsiksi `crs`-attribuutilla:

In [ ]:
municipalities.crs

Yläpuolisesta tulosteesta saamme paljon hyödyllistä tietoa: esimerkiksi sen, että geometrioihin liittyvät yksiköt ovat metrejä.

Projisointi johonkin toiseen koordinaattijärjestelmään tapahtuu `to_crs`-metodilla:

In [ ]:
municipalities_wgs = municipalities.to_crs(epsg=4326)
municipalities_wgs.crs

In [ ]:
municipalities_wgs.plot(aspect="equal")

## Harjoitus - dokumentaatio

Huomaat ylläolevassa solussa, että `plot`-funktion parametrille `aspect` annettiin arvoksi `"equal"`. Mitä tämä tarkoittaa?

1. Vinkki: Hyödynnä hakukonetta, hyvä lähtökohta hakusanalle voi olla "geodataframe plot".
2. Vinkki: Hyödynnä hakukoneen jälkeen dokumentaatiosivuja, jos sellaisia sattuu hakutulokseen ilmestymään.

Jos GeoDataFramella ei ole `crs`-attribuuttia, ei sitä voida uudelleenprojisoida. Tässä tapauksessa täytyy koordinaattijärjestelmä ensin asettaa `set_crs`-metodilla:

In [ ]:
# Poistetaan ensin tieto koordinaattijärjestelmästä
unprojected = municipalities.set_crs(None, allow_override=True)
print(unprojected.crs)
unprojected.head()

In [ ]:
# Asetetaan crs takaisin
projected = unprojected.set_crs(epsg=3067)
projected.crs.name

## Harjoitus - projektio

Projisoi kunta-aineisto (`municipalities`) johonkin haluamaasi koordinaatistoon. Voit esimerkiksi käyttää suosittua web-mercatoria, eli `epsg=3857`. Miltä Suomi nyt näyttää?

1. Tallenna uudelleenprojisoinnin tulos muuttujaan
2. Käytä `plot`-metodia

In [ ]:
## Kirjoita ratkaisu


## Ratkaisu

In [ ]:
reprojected = municipalities.to_crs(epsg=3857)
reprojected.plot()

# Geometrioiden toiminnallisuuksia

GeoDataFrame ja GeoSeries toteuttavat lukuisia geometrioiden kanssa tomimiseen tarkoitettuja ja optimoituja attribuutteja ja metodeja. Alla muutama esimerkki.

Aloitetaan laskemalla alueita ja keskipisteitä:

In [ ]:
municipalities.area

In [ ]:
centroids = municipalities.centroid
centroids.plot()

Jokaisen kunnan bounding box eli `enevelope`. Huomaa, että voimme piirtää vain ääriviivat asettamalla `plot`-metodille `facecolor="none"`.

In [ ]:
municipalities.envelope.plot(facecolor="none")

Geometrioiden yhdistäminen:

In [ ]:
dissolved = municipalities.dissolve()
dissolved.plot()

In [ ]:
dissolved

Koko GeoDataFramen sisältävä konveksi monikulmio ja bounding box:

In [ ]:
dissolved.convex_hull.plot()

In [ ]:
dissolved.envelope.plot()

geometrian yksinkertaistaminen:

In [ ]:
dissolved.simplify(tolerance=40000).plot()

Tarkastellaan lopuksi vielä muutamaa **overlay**-operaatiota.

Muodostetaan Helsingille ensin bufferialue:

In [ ]:
# Luodaan vain Helsingin sisältävä kopio kunta-aineistosta 
helsinki_buffer = municipalities.loc[municipalities["nimi"] == "Helsinki"].copy()

# Tehdään bufferi ja asetetaan se geometriaksi
# Buffer-metodille annetaan etäisyys koordinaattijärjestelmän mukaisessa yksikössä, nyt siis metreissä
helsinki_buffer["geometry"] = helsinki_buffer.centroid.buffer(150000)

Havainnollistetaan lähtötilanne visuaalisesti:

In [ ]:
ax = municipalities.plot()
helsinki_buffer.plot(ax=ax, facecolor="orange", alpha=0.5)

Itse overlay-analyysi tapahtuu `overlay`-metodilla, jonka `how`-parametrille annetaan merkkijonona overlayn tyyppi.

Valitaan yhteinen alue (**intersection**), eli annetaan metodikutsussa `how="intersection"`:

In [ ]:
intersection = municipalities.overlay(helsinki_buffer, how="intersection")
intersection.plot()

Yhdistetään geometrioita **Union**illa:

In [ ]:
dissolved.overlay(helsinki_buffer, how="union").plot()

Erotus eli **difference**:

In [ ]:
dissolved.overlay(helsinki_buffer, how="difference").plot()

**Symmetric difference**:

In [ ]:
symmetric_diff = dissolved.overlay(helsinki_buffer, how="symmetric_difference")
symmetric_diff.plot()

## Harjoitus - merialue Uusimaan edustalla

Tee yksinkertainen kartta, jossa visualisoit äsken tehdyssä `symmetric_difference` -overlayssa syntyneen merialueen.
1. Miten pääset käsiksi pelkkään merialueeseen? Käytä jo luotua `symmetric_diff` -muuttujaa.
1. Muodosta GeoDataFrame, jossa on vain haluttu merialue.
1. Käytä `plot`-metodia.

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
sea = symmetric_diff.iloc[[1]]
sea.plot()